## ETL PART 1 - This code is not for execution. It will not work, since you do not have the proper directory structure.

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import os
import calendar
import time
%matplotlib inline



In [2]:
#Stuff is hardcoded and no functions yet. Keep it simple. Do not write stuff that is hard to follow, 
#even if there is a small performance penalty. No recursion. Thank you.
#2017 data only
print("\nProcessing Data\n")
#No date or time added to log file yet
log_file = open("err_log1.log", "a")
# master list of shots for all players 
# (per tournament, per year, per player, per round, per hole)
all_rows_of_shots_list = [] 

try:
    #Stuff is hardcoded and no functions yet. Thank you.
    #tournament name lookup
    with open(".\\data1\\tourney-lookup.json", 'r') as tour_content:
        tour_data = json.load(tour_content)
    #player name lookup
    df_players = pd.read_json(".\\data1\\players.json")
    df_players.set_index('pid')
    d_base_path = r".\data1\tournaments"
    if not os.path.exists(d_base_path):
        raise NameError(d_base_path)
    else:
        #Get all tournaments
        all_tournaments = os.listdir(d_base_path)
        for this_tournament in all_tournaments:
            #Stuff is hardcoded and no functions yet. Thank you.
            d_path = ".\\data1\\tournaments\\" + this_tournament + "\\2017\\scorecards"
            if not os.path.exists(d_path):
                log_file.write("Failed to find path: {0}\n".format(d_path))
            else:
                #Get all players for this tournament
                all_players = os.listdir(d_path)
                for this_player in all_players:
                    d_file = d_path + "\\" + this_player
                    #Get data for this player
                    with open(d_file) as content_file:
                        header_data_item = "" 
                        # One row of shot data per shot
                        row_of_shot_data_list = []
                        for line in content_file:
                            json_txt = json.loads(line)
                            #Start building row header
                            #Add tournament code to row_of_shot_data
                            row_of_shot_data_list.append(this_tournament)
                            #Add tournament name to row_of_shot_data                            
                            if str(this_tournament) in tour_data.keys():
                                row_of_shot_data_list.append(tour_data[str(this_tournament)])
                            else:
                                row_of_shot_data_list.append("Tournament Name N/A for code " + str(this_tournament))                            
                            #Add tournament year to row_of_shot_data                            
                            row_of_shot_data_list.append('2017')
                            #Add player id, player full name, and player country and  to row_of_shot_data                            
                            for this_header_key in json_txt.get("p"):
                                if this_header_key != "rnds":
                                    if this_header_key != "msgId":
                                        if this_header_key == "id":
                                            player_id = json_txt.get("p")[this_header_key]
                                            row_of_shot_data_list.append(player_id)
                                            if player_id == "":
                                                player_id_int = 0
                                            else:
                                                player_id_int = int(player_id)
                                            df3 = df_players[(df_players['pid'] == player_id_int)]
                                            if df3.shape[0] <= 0:
                                                p_full_name = "Player Full Name N/A for Code " + str(player_id_int)
                                                p_country = "Player Country N/A for Code " + str(player_id_int)
                                                row_of_shot_data_list.append(p_full_name)
                                                row_of_shot_data_list.append(p_country)
                                            else:
                                                p_full_name = str(df3['nameF'].iloc[0]) + " " + str(df3['nameL'].iloc[0])
                                                p_country = str(df3['country'].iloc[0])
                                                row_of_shot_data_list.append(p_full_name)
                                                row_of_shot_data_list.append(p_country)                                                
                                        else:
                                            header_data_item = json_txt.get("p")[this_header_key]
                                            row_of_shot_data_list.append(header_data_item)
                                #Done building row header
                                #Process shot data: Find each round, 
                                #then find each hole for this round,
                                #then find each shot for this hole,
                                #add shot data to row_of_shot_data 
                                #For example,
                                #round 1, hole1, and shot 1
                                #round 1, hole 1, and shot 2
                                #round 1, hole 1, and shot n
                                #       :
                                #       :
                                #       :
                                #round 1, hole18, and shot 1
                                #round 1, hole18, and shot n
                                #       :
                                #       :
                                #       :
                                #round 2, hole n, and shot 1
                                #round 2, hole n, and shot n
                                #       :
                                #       :
                                #       :
                                #round n, hole n, and shot n
                                else:
                                    for this_round in json_txt.get("p")["rnds"]:
                                        for key in this_round:
                                            if key != "holes":
                                                row_of_shot_data_list.append(this_round[key])
                                            else:
                                                for this_hole in this_round["holes"]:
                                                    for key in this_hole:
                                                        if key != "shots":
                                                            row_of_shot_data_list.append(this_hole[key])
                                                        else:
                                                            from_x = ''
                                                            from_y = ''
                                                            for this_shot in this_hole["shots"]:
                                                                for key in this_shot:
                                                                    if key == 'x':
                                                                        row_of_shot_data_list.append(str(from_x))
                                                                        row_of_shot_data_list.append(this_shot[key])
                                                                        from_x = this_shot[key]
                                                                    elif key == 'y':
                                                                        row_of_shot_data_list.append(str(from_y))
                                                                        row_of_shot_data_list.append(this_shot[key])
                                                                        from_y = this_shot[key]
                                                                    else:
                                                                        row_of_shot_data_list.append(this_shot[key])
                                                                #Add this row shot for this hole, for this round,
                                                                #for this player, for this year, for this tournament
                                                                #to all_rows_of_shots master list 
                                                                all_rows_of_shots_list.append(row_of_shot_data_list)
                                                                #Prepare row for next shot to be added: remove
                                                                #this shot's data from row, to load next shot
                                                                row_of_shot_data_list = row_of_shot_data_list[0:12]
                                                            #loaded all shots
                                                    #Prepare row for next hole to be added: remove
                                                    #this hole's data from row, to load next hole
                                                    row_of_shot_data_list = row_of_shot_data_list[0:7]
                                                #Loaded all holes
                                        #Prepare row for next round to be added: remove
                                        #this round's data from row, to load next round
                                        row_of_shot_data_list = row_of_shot_data_list[0:6]
                                    #Loaded all rounds
                            #Clear row for next player file
                            row_of_shot_data_list = []
                #Load next player (file)
        #Load next torunament (directory)
except NameError:
    log_file.write("Could not find path: {0}\n".format(str(d_path)))
except Exception as e:
    log_file.write("Failed to finish processing: {0}\n".format(str(e)))
    print("\nFinished Processing with errors\n") 
finally:
    if not log_file.closed:
        log_file.close()
        
print("\nFinished Processing\n") 
print("\n---Main list has " + str(len(all_rows_of_shots_list)) + " records---\n")
#print(all_rows_of_shots_list)



Processing Data


Finished Processing


---Main list has 29276 records---



In [3]:
#Stuff is hardcoded and no functions yet. Keep it simple. Do not write stuff that is hard to follow, 
#even if there is a small performance penalty. No recursion. Thank you.
#2016 data only
print("\nProcessing Data\n")
#No date or time added to log file yet
log_file = open("err_log1.log", "a")
# master list of shots for all players 
# (per tournament, per year, per player, per round, per hole)
#all_rows_of_shots_list = [] 

try:
    #Stuff is hardcoded and no functions yet. Thank you.
    #tournament name lookup
    with open(".\\data1\\tourney-lookup.json", 'r') as tour_content:
        tour_data = json.load(tour_content)
    #player name lookup
    df_players = pd.read_json(".\\data1\\players.json")
    df_players.set_index('pid')
    d_base_path = r".\data1\tournaments"
    if not os.path.exists(d_base_path):
        raise NameError(d_base_path)
    else:
        #Get all tournaments
        all_tournaments = os.listdir(d_base_path)
        for this_tournament in all_tournaments:
            #Stuff is hardcoded and no functions yet. Thank you.
            d_path = ".\\data1\\tournaments\\" + this_tournament + "\\2016\\scorecards"
            if not os.path.exists(d_path):
                log_file.write("Failed to find path: {0}\n".format(d_path))
            else:
                #Get all players for this tournament
                all_players = os.listdir(d_path)
                for this_player in all_players:
                    d_file = d_path + "\\" + this_player
                    #Get data for this player
                    with open(d_file) as content_file:
                        header_data_item = "" 
                        # One row of shot data per shot
                        row_of_shot_data_list = []
                        for line in content_file:
                            json_txt = json.loads(line)
                            #Start building row header
                            #Add tournament code to row_of_shot_data
                            row_of_shot_data_list.append(this_tournament)
                            #Add tournament name to row_of_shot_data                            
                            if str(this_tournament) in tour_data.keys():
                                row_of_shot_data_list.append(tour_data[str(this_tournament)])
                            else:
                                row_of_shot_data_list.append("Tournament Name N/A for code " + str(this_tournament))                            
                            #Add tournament year to row_of_shot_data                            
                            row_of_shot_data_list.append('2016')
                            #Add player id, player full name, and player country and  to row_of_shot_data                            
                            for this_header_key in json_txt.get("p"):
                                if this_header_key != "rnds":
                                    if this_header_key != "msgId":
                                        if this_header_key == "id":
                                            player_id = json_txt.get("p")[this_header_key]
                                            row_of_shot_data_list.append(player_id)
                                            if player_id == "":
                                                player_id_int = 0
                                            else:
                                                player_id_int = int(player_id)
                                            df3 = df_players[(df_players['pid'] == player_id_int)]
                                            if df3.shape[0] <= 0:
                                                p_full_name = "Player Full Name N/A for Code " + str(player_id_int)
                                                p_country = "Player Country N/A for Code " + str(player_id_int)
                                                row_of_shot_data_list.append(p_full_name)
                                                row_of_shot_data_list.append(p_country)
                                            else:
                                                p_full_name = str(df3['nameF'].iloc[0]) + " " + str(df3['nameL'].iloc[0])
                                                p_country = str(df3['country'].iloc[0])
                                                row_of_shot_data_list.append(p_full_name)
                                                row_of_shot_data_list.append(p_country)                                                
                                        else:
                                            header_data_item = json_txt.get("p")[this_header_key]
                                            row_of_shot_data_list.append(header_data_item)
                                #Done building row header
                                #Process shot data: Find each round, 
                                #then find each hole for this round,
                                #then find each shot for this hole,
                                #add shot data to row_of_shot_data 
                                #For example,
                                #round 1, hole1, and shot 1
                                #round 1, hole 1, and shot 2
                                #round 1, hole 1, and shot n
                                #       :
                                #       :
                                #       :
                                #round 1, hole18, and shot 1
                                #round 1, hole18, and shot n
                                #       :
                                #       :
                                #       :
                                #round 2, hole n, and shot 1
                                #round 2, hole n, and shot n
                                #       :
                                #       :
                                #       :
                                #round n, hole n, and shot n
                                else:
                                    for this_round in json_txt.get("p")["rnds"]:
                                        for key in this_round:
                                            if key != "holes":
                                                row_of_shot_data_list.append(this_round[key])
                                            else:
                                                for this_hole in this_round["holes"]:
                                                    for key in this_hole:
                                                        if key != "shots":
                                                            row_of_shot_data_list.append(this_hole[key])
                                                        else:
                                                            from_x = ''
                                                            from_y = ''
                                                            for this_shot in this_hole["shots"]:
                                                                for key in this_shot:
                                                                    if key == 'x':
                                                                        row_of_shot_data_list.append(str(from_x))
                                                                        row_of_shot_data_list.append(this_shot[key])
                                                                        from_x = this_shot[key]
                                                                    elif key == 'y':
                                                                        row_of_shot_data_list.append(str(from_y))
                                                                        row_of_shot_data_list.append(this_shot[key])
                                                                        from_y = this_shot[key]
                                                                    else:
                                                                        row_of_shot_data_list.append(this_shot[key])
                                                                #Add this row shot for this hole, for this round,
                                                                #for this player, for this year, for this tournament
                                                                #to all_rows_of_shots master list 
                                                                all_rows_of_shots_list.append(row_of_shot_data_list)
                                                                #Prepare row for next shot to be added: remove
                                                                #this shot's data from row, to load next shot
                                                                row_of_shot_data_list = row_of_shot_data_list[0:12]
                                                            #loaded all shots
                                                    #Prepare row for next hole to be added: remove
                                                    #this hole's data from row, to load next hole
                                                    row_of_shot_data_list = row_of_shot_data_list[0:7]
                                                #Loaded all holes
                                        #Prepare row for next round to be added: remove
                                        #this round's data from row, to load next round
                                        row_of_shot_data_list = row_of_shot_data_list[0:6]
                                    #Loaded all rounds
                            #Clear row for next player file
                            row_of_shot_data_list = []
                #Load next player (file)
        #Load next torunament (directory)
except NameError:
    log_file.write("Could not find path: {0}\n".format(str(d_path)))
except Exception as e:
    log_file.write("Failed to finish processing: {0}\n".format(str(e)))
    print("\nFinished Processing with errors\n") 
finally:
    if not log_file.closed:
        log_file.close()
        
print("\nFinished Processing\n") 
print("\n---Main list has " + str(len(all_rows_of_shots_list)) + " records---\n")
#print(all_rows_of_shots_list)



Processing Data


Finished Processing


---Main list has 58005 records---



In [4]:
#Stuff is hardcoded and no functions yet. Keep it simple. Do not write stuff that is hard to follow, 
#even if there is a small performance penalty. No recursion. Thank you.
#2015 data only
print("\nProcessing Data\n")
#No date or time added to log file yet
log_file = open("err_log1.log", "a")
# master list of shots for all players 
# (per tournament, per year, per player, per round, per hole)
#all_rows_of_shots_list = [] 

try:
    #Stuff is hardcoded and no functions yet. Thank you.
    #tournament name lookup
    with open(".\\data1\\tourney-lookup.json", 'r') as tour_content:
        tour_data = json.load(tour_content)
    #player name lookup
    df_players = pd.read_json(".\\data1\\players.json")
    df_players.set_index('pid')
    d_base_path = r".\data1\tournaments"
    if not os.path.exists(d_base_path):
        raise NameError(d_base_path)
    else:
        #Get all tournaments
        all_tournaments = os.listdir(d_base_path)
        for this_tournament in all_tournaments:
            #Stuff is hardcoded and no functions yet. Thank you.
            d_path = ".\\data1\\tournaments\\" + this_tournament + "\\2015\\scorecards"
            if not os.path.exists(d_path):
                log_file.write("Failed to find path: {0}\n".format(d_path))
            else:
                #Get all players for this tournament
                all_players = os.listdir(d_path)
                for this_player in all_players:
                    d_file = d_path + "\\" + this_player
                    #Get data for this player
                    with open(d_file) as content_file:
                        header_data_item = "" 
                        # One row of shot data per shot
                        row_of_shot_data_list = []
                        for line in content_file:
                            json_txt = json.loads(line)
                            #Start building row header
                            #Add tournament code to row_of_shot_data
                            row_of_shot_data_list.append(this_tournament)
                            #Add tournament name to row_of_shot_data                            
                            if str(this_tournament) in tour_data.keys():
                                row_of_shot_data_list.append(tour_data[str(this_tournament)])
                            else:
                                row_of_shot_data_list.append("Tournament Name N/A for code " + str(this_tournament))                            
                            #Add tournament year to row_of_shot_data                            
                            row_of_shot_data_list.append('2015')
                            #Add player id, player full name, and player country and  to row_of_shot_data                            
                            for this_header_key in json_txt.get("p"):
                                if this_header_key != "rnds":
                                    if this_header_key != "msgId":
                                        if this_header_key == "id":
                                            player_id = json_txt.get("p")[this_header_key]
                                            row_of_shot_data_list.append(player_id)
                                            if player_id == "":
                                                player_id_int = 0
                                            else:
                                                player_id_int = int(player_id)
                                            df3 = df_players[(df_players['pid'] == player_id_int)]
                                            if df3.shape[0] <= 0:
                                                p_full_name = "Player Full Name N/A for Code " + str(player_id_int)
                                                p_country = "Player Country N/A for Code " + str(player_id_int)
                                                row_of_shot_data_list.append(p_full_name)
                                                row_of_shot_data_list.append(p_country)
                                            else:
                                                p_full_name = str(df3['nameF'].iloc[0]) + " " + str(df3['nameL'].iloc[0])
                                                p_country = str(df3['country'].iloc[0])
                                                row_of_shot_data_list.append(p_full_name)
                                                row_of_shot_data_list.append(p_country)                                                
                                        else:
                                            header_data_item = json_txt.get("p")[this_header_key]
                                            row_of_shot_data_list.append(header_data_item)
                                #Done building row header
                                #Process shot data: Find each round, 
                                #then find each hole for this round,
                                #then find each shot for this hole,
                                #add shot data to row_of_shot_data 
                                #For example,
                                #round 1, hole1, and shot 1
                                #round 1, hole 1, and shot 2
                                #round 1, hole 1, and shot n
                                #       :
                                #       :
                                #       :
                                #round 1, hole18, and shot 1
                                #round 1, hole18, and shot n
                                #       :
                                #       :
                                #       :
                                #round 2, hole n, and shot 1
                                #round 2, hole n, and shot n
                                #       :
                                #       :
                                #       :
                                #round n, hole n, and shot n
                                else:
                                    for this_round in json_txt.get("p")["rnds"]:
                                        for key in this_round:
                                            if key != "holes":
                                                row_of_shot_data_list.append(this_round[key])
                                            else:
                                                for this_hole in this_round["holes"]:
                                                    for key in this_hole:
                                                        if key != "shots":
                                                            row_of_shot_data_list.append(this_hole[key])
                                                        else:
                                                            from_x = ''
                                                            from_y = ''
                                                            for this_shot in this_hole["shots"]:
                                                                for key in this_shot:
                                                                    if key == 'x':
                                                                        row_of_shot_data_list.append(str(from_x))
                                                                        row_of_shot_data_list.append(this_shot[key])
                                                                        from_x = this_shot[key]
                                                                    elif key == 'y':
                                                                        row_of_shot_data_list.append(str(from_y))
                                                                        row_of_shot_data_list.append(this_shot[key])
                                                                        from_y = this_shot[key]
                                                                    else:
                                                                        row_of_shot_data_list.append(this_shot[key])
                                                                #Add this row shot for this hole, for this round,
                                                                #for this player, for this year, for this tournament
                                                                #to all_rows_of_shots master list 
                                                                all_rows_of_shots_list.append(row_of_shot_data_list)
                                                                #Prepare row for next shot to be added: remove
                                                                #this shot's data from row, to load next shot
                                                                row_of_shot_data_list = row_of_shot_data_list[0:12]
                                                            #loaded all shots
                                                    #Prepare row for next hole to be added: remove
                                                    #this hole's data from row, to load next hole
                                                    row_of_shot_data_list = row_of_shot_data_list[0:7]
                                                #Loaded all holes
                                        #Prepare row for next round to be added: remove
                                        #this round's data from row, to load next round
                                        row_of_shot_data_list = row_of_shot_data_list[0:6]
                                    #Loaded all rounds
                            #Clear row for next player file
                            row_of_shot_data_list = []
                #Load next player (file)
        #Load next torunament (directory)
except NameError:
    log_file.write("Could not find path: {0}\n".format(str(d_path)))
except Exception as e:
    log_file.write("Failed to finish processing: {0}\n".format(str(e)))
    print("\nFinished Processing with errors\n") 
finally:
    if not log_file.closed:
        log_file.close()
        
print("\nFinished Processing\n") 
print("\n---Main list has " + str(len(all_rows_of_shots_list)) + " records---\n")
#print(all_rows_of_shots_list)



Processing Data


Finished Processing


---Main list has 87637 records---



In [5]:
#Send data to frame for csv output, analysis, etc.
my_cols = ["tournament",
           "tournament_name",
           "year",
           "id",
           "p_full_name",
           "p_country",
           "rnds",
           "holes",
           "cNum",
           "sc",
           "pDay",
           "pTot",
           "shots",
           "pid",
           "time",
           "putt",
           "t",
           "prv",
           "tee",
           "cup",
           "from",
           "to",
           "asc",
           "dist",
           "left",
           "from_x",
           "x",
           "from_y",
           "y",
           "z",
           "club",
           "con",
           "shotText"]

df = pd.DataFrame(all_rows_of_shots_list,columns=my_cols)
#df.set_index("id")
#df.tail()
#print(str(df.shape[0]))
df.to_csv("Waste_Mgt_Phoenix_Only_2015_2016_2017.csv", mode='w', index=False)


# END 